<a href="https://colab.research.google.com/github/vuchicago/Python/blob/master/DSO_monthly_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


####CODE WRITTEN @VUNGUYEN UPDATED MAY 29, 2019


####BEFORE YOU INITIALIZE THE CODE, PLEASE GO THROUGH THESE 5 INITIAL STEPS.  


1) Go to google drive to "Shared with me" folder.  Right click on "DSO For SalesForce" folder that has been shared.  
   Select "Add to My Drive" - YOU ONLY NEED TO DO THIS ONCE. IF ALREADY Done IN PREVIOUS MONTHS, THEN SKIP.
   
2) Download all the dso files to inspect.  
       
       a) Make sure that all of the rows and columns are in the SAME EXACT order as the previous month.  
                *For example, Column A or B should be the same fields, even if blank. Sometimes people will delete a column and this will produce a code error
  
        b) Make sure all of the tabs contain data.  If there is a tab in the file that contains no data, then DELETE the tab

3) Create a folder called "Sales Districts" within the main folder for that month.  
        
        a) Drag all the files that have only only one district or group into this folder.  You can tell by which files contain      "DBAC_REPORT" as a tab      
      
        b) leave all others that contain multiple districts alone (Generally Midwest, North Central, Northeast, Southern, Inside Sales)


4) Mount Drive using code below.  Click Shift + Enter.  Or Go to Runtime -> Run Selection

5) Update all of the relevant fields below the mount code that says "UPDATE MONTHLY" 


In [64]:

####MOUNT DRIVE
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [65]:
import os, shutil, pandas as pd, numpy as np

############################################################# UPDATE MONTHLY   ############################################
year='2019' ###UPDATE ONCE PER YEAR

dso_output_file="dso_May2019.xlsx" ####UPDATE MONTHLY

dso_main_folder="May (April data)"  ####UPDATE MONTHLY

#######            UPDATE MONTHLY
midwest="DBAC  4-30-19  Midwest Region (1000).xlsx" 
ncentral='DBAC  4-30-19  North Central (2500).xlsx'
inside='DBAC  4-30-19  Inside Sales (8500).xlsx'
southern='SR 3000 DSO April 2019.xlsx'
northeast='DBAC Northeast for Salesforce 4-30-19.xlsx'


############################################################# DONE WITH UPDATE.  DON'T TOUCH CODE BELOW       ##################################
#########################################################################################################################################

####DIRECTORIES OF FILES

dso_dir='/content/gdrive/My Drive/DSO For Salesforce' ##original dso directory
base_dir=os.path.join(dso_dir,year) 
main_dir=os.path.join(base_dir,dso_main_folder) 
adl_dir=os.path.join(main_dir,"Sales Districts")

print("Files in Main folder:",os.listdir(base_dir))
print("Number of files in Sales District Subfolder:",len(os.listdir(adl_dir)))



Files in Main folder: ['April (March Data)', 'May (April data)', 'March (Feb data)', 'Cement User ID_SFDC.xlsx', 'dso_May2019.xlsx']
Number of files in Sales District Subfolder: 19


In [66]:



#######THE LIST OF DISTRICT MANAGERS
os.chdir(base_dir)                                                               # change directory to base directory
district_managers=pd.read_excel("Cement User ID_SFDC.xlsx",sheet_name="Manager")  ###read the district manager list
district_managers["Sales District Text"]=[str(i) for i in district_managers["Sales District Text"]]
district_managers["USER_C"]=district_managers["User ID"]
col_keep=["Sales District Text","User ID","USER_C"]
district_managers=district_managers[col_keep].rename(columns={"User ID":"OWNERID"})
############################################

os.chdir(main_dir)
###Dictionary to map month given to column "PERIOD__C" in the dso upload file
month={"Jan":1,'Feb':2,'Mar':3,'Apr':4,'May':5,"Jun":6,"Jul":7,"Aug":8, "Sep":9,"Oct":10,"Nov":11,"Dec":12}
month_rev = dict([[v,k] for k,v in month.items()]) #reverse dictionary numbered months: 3 letter months

#####Dictionary to map sales office number to "SALES_OFFICE__C" in the dso upload file
sales_office={'1000':"1000 - Midwest",'2000, 4000':"2000, 4000 - West",'2500':"2500 - North Central",'3000':"3000 - Southern",
              '4000':"4000 - TX/OK",'7000':"7000 - Northeast",'8500':"8500 - Inside Sales", '9000':"9000 - Trade Sales"}

print("Number of Sales Offices:" ,len(sales_office))



Number of Sales Offices: 8


In [0]:
#####CREATE OUTPUT_FN TO READ IN THE TOP ROWS 3-9 in DBAC files sent by regions

def output_top(input_files,i):
    output=pd.DataFrame()
    #####PULLS DATA FROM ROWS 3 TO 9 IN THE DBAC FILES###############
    dbac_info=pd.read_excel(input_files,sheet_name=i,skiprows=1,index_cols=None,na_values="NA").iloc[:7] ##top portion of dbac file
    dbac_info.columns=[i for i,j in enumerate(dbac_info.columns)] ###GRAB and NAME ALL THE COLUMNS
    
    ###CREATE SEPARATE DATAFRAMES TO GRAB ONLY 4 COLUMNS.  
    temp1=dbac_info[[1,3]] ##COLUMN 1,3,11,13 HAS ALL RELEVANT DATA
    temp2=dbac_info[[11,13]] 
    temp1.columns=[i for i,j in enumerate(temp1)]  ##GIVE THE COLUMNS ARBITARY NAMES
    temp2.columns=[i for i,j in enumerate(temp2)]
    dbac_info=pd.concat([temp1,temp2])
    dbac_info.index=dbac_info[0]
    ####DEFINE SALES GROUP####
    output['filler1']="A"  
    output['filler2']=dbac_info.loc["Key Calendar Year"] 
    output['filler3']="Total (10 + 12)"  
    output['filler4']='N/A'
    if (dbac_info.loc["Sales Group"][1]=="Empty Demarcation") & (dbac_info.loc["Sales District"][1]=="Empty Demarcation"):
        output["REPORT_LEVEL__C"]="Sales Office"
    elif (dbac_info.loc["Sales Office"][1]!="Empty Demarcation") &(dbac_info.loc["Sales District"][1]=="Empty Demarcation"):
        output["REPORT_LEVEL__C"]="Sales Group"
    else:
        output["REPORT_LEVEL__C"]="Sales District"
    
    output["SALES_OFFICE__C"]=sales_office[dbac_info.loc["Sales Office"][1]] ##convert sales office number to num+key using dictionary above
        #####DEFINE SALES DISTRICT     
    if dbac_info.loc["Sales District"][1]=="Empty Demarcation":
        output["SALES_DISTRICT__C"]=" "
    else:
        output["SALES_DISTRICT__C"]=dbac_info.loc["Sales District"]
    
    if dbac_info.loc["Sales Group"][1]=="Empty Demarcation":
        output["SALES_GROUP__C"]=" "
    else:
        output["SALES_GROUP__C"]=dbac_info.loc["Sales Group"]    

    output['joincol']=1 ###CREATE PRIMARY KEY TO JOIN WITH DBAC_  
    output["YEAR__C"]=dbac_info.loc["Key Calendar Year"]
    output["DIVISION__C"]="Total (10 + 12)" 
    return output.iloc[1:2]


In [0]:
####READS THE BOTTOM HALF OF THE DSO INPUT FILE

def output_bot(input_files,i):
    dbac_bot=pd.read_excel(input_files,sheet_name=i,skiprows=11,index_col=1,na_values="NA").iloc[:12]
    dbac_bot['joincol']=1 ##joincol will act as the primary key
    dbac_bot=dbac_bot[dbac_bot["EOM A/R 1"]!=0]  ###This reads in values in files wheree it's not filler
    dbac_bot["PERIOD__C"]="" ##CREATE PERIOD_C
    for i in range(len(dbac_bot)):
        dbac_bot["PERIOD__C"].iloc[i]=month[dbac_bot.index[i]]
    return dbac_bot

#####MERGES ALL OF THE BOTTOM AND TOP HALF DATAFRAMES CREATED PREVIOUSLY.  IF THERE ARE 
###MULTIPLE TABS, THEN ALL THE ONES GENERATED FROM MULTIPLE TABS ARE MERGED TOGETHER
def dso_salesforce(input_files,tabs):
    output_files=pd.DataFrame()
    
    for i in tabs: 
        ####READ IN FILE AND CREATE MONTH VARIABLE FOR OUTPUT
        dbac_top=output_top(input_files,i) ###USE OUTPUT_FN HERE
        dbac_bot=output_bot(input_files,i)
        dbac_bot=dbac_bot.reset_index()

        output_file=dbac_top.merge(dbac_bot,how='left',on='joincol',suffixes=('_top','_dbac'))  ####MERGE THE BOT AND TOP OF DBAC FILE
        output_file=output_file.rename(columns={"Sales Office":"SALES_OFFICE__C",
                                                "BPDSO 4":"BPDSO__C","DSO 3":"DSO__C",
                                                '12 Month Rolling\nAverage DSO':"ROLLING_DSO__C"})      
    ###CREATE LEVEL VARIABLE IN ORDER TO MATCH IT WITH THE SALES DISTRICT IN THE Cement User ID_SFDC file    
        output_file['Level']=""
        for i in range(output_file.shape[0]):
            if (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales Office"):
                output_file["Level"].iloc[i]=output_file["SALES_OFFICE__C"].iloc[i]
            elif (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales District"):
                output_file["Level"].iloc[i]=output_file["SALES_DISTRICT__C"].iloc[i]
            else:
                output_file["Level"].iloc[i]=output_file["SALES_GROUP__C"].iloc[i]       
        columns=["Level","REPORT_LEVEL__C","SALES_OFFICE__C","SALES_GROUP__C","SALES_DISTRICT__C","PERIOD__C","YEAR__C","DIVISION__C","BPDSO__C","DSO__C","ROLLING_DSO__C"]   
        output_file=output_file[columns]
        
        output_files=output_files.append(output_file,ignore_index=True).dropna()
    return output_files

In [0]:
#####THIS CONSONSOLIDATES FILES THAT ONLY HAVE ONE DISTRICT/GROUP.  THIS READS IT IN THE "Sales Districts" Folder

def dso_salesforce_unconsol(folder,tabs):
    output_files=pd.DataFrame()
    import os
    os.chdir(folder)
    input_files=os.listdir(folder)
    for input_file in input_files:
        for i in tabs:
            
        #input_file=xl.parse(i)  # read a specific sheet to DataFrame    
        ####READ IN FILE AND CREATE MONTH VARIABLE FOR OUTPUT
            dbac_top=output_top(input_file,i) ###USE OUTPUT_FN HERE
            dbac_bot=output_bot(input_file,i)
            dbac_bot=dbac_bot.reset_index()
            
            output_file=dbac_top.merge(dbac_bot,how='left',on='joincol',suffixes=('_top','_dbac'))  ####MERGE THE BOT AND TOP OF DBAC FILE
            output_file=output_file.rename(columns={"Sales Office":"SALES_OFFICE__C",  "BPDSO 4":"BPDSO__C","DSO 3":"DSO__C",
                                                    '12 Month Rolling\nAverage DSO':"ROLLING_DSO__C"})        
            ###CREATE LEVEL VARIABLE IN ORDER TO MATCH IT WITH THE SALES DISTRICT IN THE Cement User ID_SFDC file    
            output_file["Level"]=""
            for i in range(output_file.shape[0]):
                if (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales Office"):
                    output_file["Level"].iloc[i]=output_file["SALES_OFFICE__C"].iloc[i]
                elif (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales District"):
                    output_file["Level"].iloc[i]=output_file["SALES_DISTRICT__C"].iloc[i]
                else:
                    output_file["Level"].iloc[i]=output_file["SALES_GROUP__C"].iloc[i]                  
            ####output the file and append it to the previous output file
            columns=["Level","REPORT_LEVEL__C","SALES_OFFICE__C","SALES_GROUP__C","SALES_DISTRICT__C","PERIOD__C","YEAR__C","DIVISION__C","BPDSO__C","DSO__C","ROLLING_DSO__C"]   
            output_file=output_file[columns]
        
        output_files=output_files.append(output_file,ignore_index=True).dropna()
    return output_files

def west_salesgroup(dset_in):
    for i in range(len(dset_in)):
        if dset_in["SALES_DISTRICT__C"].iloc[i] in ('2314','2317','2319','2315','2230','2219','2212','2213'):
            dset_in["SALES_OFFICE__C"].iloc[i]="2000 - Mountain"
        if dset_in["SALES_DISTRICT__C"].iloc[i] in ('2911','2937', '2936', '2932','2933','2989','2938','2913','2939','2934','2935'):
            dset_in["SALES_OFFICE__C"].iloc[i]="4000 - TX/OK"
        else:
            dset_in['SALES_OFFICE__C'].iloc[i]=dset_in["SALES_OFFICE__C"].iloc[i]
        if dset_in["SALES_GROUP__C"].iloc[i]=='230':
            dset_in["SALES_OFFICE__C"].iloc[i]="2000 - Mountain"
        if dset_in["SALES_GROUP__C"].iloc[i]=='490':
            dset_in["SALES_OFFICE__C"].iloc[i]="4000 - TX/OK"
        else:
            dset_in['SALES_OFFICE__C'].iloc[i]=dset_in["SALES_OFFICE__C"].iloc[i]            
    return dset_in

In [0]:
#####GRABS ALL THE NAMES OF THE TABS IN THE FILE.  EXCLUDES UNNECESSARY TABS

os.chdir(main_dir)
def tab_names(region):
    xl = pd.ExcelFile(region)
    tabs_all=xl.sheet_names  # see all sheet names
    tabs_excl=['Table',
           'DSO_Calc',
           'DaysCostCY',
           'SAPBEXqueries',
           'SAPBEXfilters',
           'DaysCostLY',
           'GrossBalCY',
           'GB_LY',
           'CurRecCY',
           'GB_2Y',
           'Global']
    tabs_incl=[tabs for tabs in tabs_all if tabs not in tabs_excl]
    return tabs_incl

In [71]:
###Excludes some non-relevant tabs
midwest_tabs=tab_names(midwest)
ncentral_tabs=tab_names(ncentral)
northeast_tabs=tab_names(northeast)
inside_tabs=tab_names(inside)
southern_tabs=tab_names(southern)



#####call the files 
df_midwest=dso_salesforce(midwest,midwest_tabs)
df_ncentral=dso_salesforce(ncentral,ncentral_tabs)
df_inside=dso_salesforce(inside,inside_tabs)
df_northeast=dso_salesforce(northeast,northeast_tabs)
df_southern=dso_salesforce(southern,southern_tabs)

###WEST AND SOUTHERN ARE IN DIFFERENT DIRECTORIES
os.chdir(adl_dir)

west_tabs=['DBAC_REPORT']


df_west=dso_salesforce_unconsol(adl_dir,west_tabs)
df_mountain_txok=west_salesgroup(df_west)


df_total=pd.concat([df_midwest,df_ncentral,df_inside,df_mountain_txok,df_southern])
col=["SALES_OFFICE__C","SALES_DISTRICT__C","SALES_GROUP__C"]

df_final=pd.merge(df_total,district_managers,left_on="Level",right_on="Sales District Text",how='left')
final_col=["USER_C","OWNERID","REPORT_LEVEL__C","SALES_OFFICE__C","SALES_GROUP__C","SALES_DISTRICT__C","PERIOD__C","YEAR__C","DIVISION__C","BPDSO__C","DSO__C","ROLLING_DSO__C","Level"]
df_final=df_final[final_col]
os.chdir(base_dir)    
df_final.to_excel(dso_output_file)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
